In [ ]:
from simulation_wrapper import simulation_wrapper_func
from sbi.analysis import ActiveSubspace, pairplot
from torch.distributions import MultivariateNormal
from sbi.simulators import linear_gaussian
from sbi.utils import BoxUniform
from sbi.inference import SNPE, simulate_for_sbi, prepare_for_sbi, infer
import time as tm
import torch

prior_min = [2.6,17.6, 0.05, 0.025] #17.6
prior_max = [9.75,110.0, 0.15, 0.075]
prior_jr = BoxUniform(low=torch.as_tensor(prior_min), high=torch.as_tensor(prior_max))

In [ ]:
posterior = infer(simulation_wrapper_func, prior_jr, num_simulations=100000, num_workers=128,
                 method="SNPE").set_default_x(torch.zeros(1000))

In [ ]:
posterior_samples = posterior.sample((1000000,))

pairplot(posterior_samples, limits=[[2.6,9.75], [17.6,110.0], [0.05,0.15], [0.025,0.075]], figsize=(4, 4), labels=[r"$A$", r"$B$", r"$a$", r"$b$"])

In [ ]:
sensitivity = ActiveSubspace(posterior)
e_vals, e_vecs = sensitivity.find_directions(posterior_log_prob_as_property=True)

print("Eigenvalues: \n", e_vals, "\n")
print("Eigenvectors: \n", e_vecs)

projected_data = sensitivity.project(posterior_samples, num_dimensions=1)